# Introduction
## What we are trying to accomplish
In general we are trying to provide answers to the following questions:
- How is the adoption of renewable energy in Europe evolving?
- How is the adoption of different types of renewable energy evolving?
- Are there characteristics of countries that correlate with their level of adoption of renewable energy?

### Main Questions
As these questions are defined very broadly and can be answered in a variety of ways, we decided to formulate the following more discreet questions, focusing on the production and consumption of renewable energy in europe:
- How has the adoption of renewable energy in Europe developed over time?
- Which types of renewable energy have been predominantly adopted?

### Sub Questions
- How do economic factors like GDP influence the adoption of renewable energy?


## Data Description
We are using the dataset from [Kaggle](https://www.kaggle.com/datasets/programmerrdai/renewable-energy/data). The general description can be found at the original [ourworldindata.org](https://ourworldindata.org/grapher/installed-solar-pv-capacity#:~:text=IRENA%20%E2%80%93%20Renewable%20Capacity-,Statistics) description.
**Absolute Values:** Absolute measurements are represented in megawatts (MW) rounded to the nearest one megawatt, with figures between zero and 0.5MW shown as a 0.
**Shares:** Shares are represented as percentages

## Load and check data
Load all required packages and modules

In [37]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

The dataset contains a wide range of data from different countries and time ranges. In this exercise we chose to only consider **European** countries and the **years (1990-2020)** to answer our questions.

In [38]:
# ISO codes for European countries
country_codes = (
    'AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA',
    'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT',
    'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'GBR')
time_window = (1990, 2020)

We clean the data by:
- Handling all NaN by dropping them
- Removing all non-European countries
- Only consider our targeted time window (1990-2020)

In [39]:
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna()  # Remove NaN values, TODO: check if this is the best option

    # Only Use European Countries & Europe accumulated data
    df = df[(df['Code'].isin(country_codes)) | (df['Entity'] == 'Europe')]

    # Only consider data from (1990-2020)
    if 'Year' in df.columns:
        df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
        df = df[(df['Year'] >= time_window[0]) & (df['Year'] <= time_window[1])]

    df.reset_index(drop=True, inplace=True)

    return df

Put it all together and read and clean the dataframes:

In [40]:
def read_and_clean_csv(file) -> pd.DataFrame:
    df = pd.read_csv(file)
    df = clean_df(df)
    return df

folder_path = os.path.join('..', 'data')
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

dataframes = {os.path.basename(file).split('.')[0]: clean_df(read_and_clean_csv(file)) for file in csv_files}

print(dataframes['share-electricity-solar'].head())
print(dataframes['share-electricity-wind'].head())

    Entity Code  Year  Solar (% electricity)
0  Austria  AUT  1990                    0.0
1  Austria  AUT  1991                    0.0
2  Austria  AUT  1992                    0.0
3  Austria  AUT  1993                    0.0
4  Austria  AUT  1994                    0.0
    Entity Code  Year  Wind (% electricity)
0  Austria  AUT  1990                   0.0
1  Austria  AUT  1991                   0.0
2  Austria  AUT  1992                   0.0
3  Austria  AUT  1993                   0.0
4  Austria  AUT  1994                   0.0


### Merge semantically relevant dataframes
Now we merge dataframes that contain semantically relevant data into one. These are:
- `share_energy:` merged from `hydro-share-energy`,`solar-share-energy`,`wind-share-energy`,`renewable-share-energy`
- `share_electricity:` merged from `share-electricity-hydro`,`share-electricity-solar`,`share-electricity-solar`,`share-electricity-wind`,`share-electricity-renewables`



In [41]:
share_energy_keys = [key for key in dataframes.keys() if 'share-energy' in key]
share_electricity_keys = [key for key in dataframes.keys() if 'share-electricity' in key]

share_energy = dataframes[share_energy_keys[0]]

for key in share_energy_keys[1:]:
    share_energy = share_energy.merge(
        dataframes[key],
        on=['Entity', 'Code', 'Year'],
        how='outer',
    )

share_electricity = dataframes[share_electricity_keys[0]]

for key in share_electricity_keys[1:]:
    share_electricity = share_electricity.merge(
        dataframes[key],
        on=['Entity', 'Code', 'Year'],
        how='outer',
    )

These newly merged frames can now be converted to a wide multi-index dataformat.

In [42]:
# TODO multi-index data

### Check the Data

In [43]:
# Display merged data
print("Merged 'share_energy' DataFrame:")
print(share_energy.head())

print("\nMerged 'share_electricity' DataFrame:")
print(share_electricity.head())

Merged 'share_energy' DataFrame:
    Entity Code  Year  Hydro (% sub energy)  Renewables (% sub energy)  \
0  Austria  AUT  1990                26.600                     27.586   
1  Austria  AUT  1991                25.290                     26.299   
2  Austria  AUT  1992                28.540                     29.655   
3  Austria  AUT  1993                29.555                     30.682   
4  Austria  AUT  1994                28.912                     29.914   

   Solar (% sub energy)  Wind (% sub energy)  
0                 0.000                  0.0  
1                 0.000                  0.0  
2                 0.000                  0.0  
3                 0.001                  0.0  
4                 0.001                  0.0  

Merged 'share_electricity' DataFrame:
    Entity Code  Year  Hydro (% electricity)  Renewables (% electricity)  \
0  Austria  AUT  1990                 63.915                      66.207   
1  Austria  AUT  1991                 62.654     

In [44]:
# Optionally, print the cleaned data for verification
for key, df in dataframes.items():
    print(f'{key}:')
    print(df.head(), '\n')

biofuel-production:
    Entity Code  Year  Biofuels Production - TWh - Total
0  Austria  AUT  1990                           0.073269
1  Austria  AUT  1991                           0.080247
2  Austria  AUT  1992                           0.089551
3  Austria  AUT  1993                           0.101181
4  Austria  AUT  1994                           0.067454 

cumulative-installed-wind-energy-capacity-gigawatts:
    Entity Code  Year  Wind Capacity
0  Austria  AUT  1997          0.020
1  Austria  AUT  1998          0.030
2  Austria  AUT  1999          0.042
3  Austria  AUT  2000          0.050
4  Austria  AUT  2001          0.067 

hydro-share-energy:
    Entity Code  Year  Hydro (% sub energy)
0  Austria  AUT  1990                26.600
1  Austria  AUT  1991                25.290
2  Austria  AUT  1992                28.540
3  Austria  AUT  1993                29.555
4  Austria  AUT  1994                28.912 

hydropower-consumption:
    Entity Code  Year  Electricity from hydro (TW